In [1]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install chroma-hnswlib langchain-chroma

  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
Using cached chromadb-0.5.23-py3-none-any.whl (628 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.6.3
    Uninstalling chromadb-0.6.3:
      Successfully uninstalled chromadb-0.6.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-chroma 0.3.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.6 which is incompatible.
open-webui 0.3.35 requires aiohttp==3.10.8, but you have aiohttp 3.11.11 which is incompatible.
open-webui 0.3.35 requires boto3==1.35.0, but you have boto3 1.34.69 which is incompatible.
open-webui 0.3.35 requires chromadb==0.5.9, but you have chromadb 0.5.23 which is incompatible.
open-webui 0.3.35 requires google-generativeai==0.7.2, but you have google-generativeai 0.5.4 which is incompatible.
open-webui 0.3.35 requires langchain==0.2.15, but you have langchain 0.3.13 which is incompatible.
open-webui 0.3.35 requires langchain-community==0.2.12, but you have langchain-community 0.3.13 which is incompatible.
open-webui 0.3.35 requires pypdf==4.3.1, but you have pypdf 5.1.

In [3]:
from langchain.vectorstores import Chroma

from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    #model="llama3.2:latest",
    model="llama3.2:latest",
    base_url="http://localhost:11434"  # Ensure Ollama is running locally
)

# Create Chroma vector store

vector_store = Chroma.from_texts(["doc1", "doc2"], embeddings, persist_directory="./chroma_db")

# Persist data to disk
vector_store.persist()

# Reload persisted data
new_vector_store = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)


C:\Users\Muthu\AppData\Local\Temp\ipykernel_21328\549446003.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()
C:\Users\Muthu\AppData\Local\Temp\ipykernel_21328\549446003.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  new_vector_store = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
